In [ ]:
repo_root = "/Users/miskodzamba/Dropbox/research/gits/spf/"
repo_root = "/home/mouse9911/gits/spf"
import sys

if repo_root not in sys.path:
    sys.path.append(repo_root)  # go to parent dir

In [ ]:
from torch.utils.data import Dataset
import yaml
import torch
from spf.rf import precompute_steering_vectors
from spf.utils import zarr_open_from_lmdb_store
from spf.dataset.v5_data import v5rx_f64_keys, v5rx_2xf64_keys
import numpy as np
from spf.rf import speed_of_light
from multiprocessing.pool import ThreadPool
import os
import pickle
from multiprocessing import Pool
import time
import tqdm

#         return pickle.load(open(results_fn, "rb"))

In [ ]:
from spf.dataset.spf_dataset import v5spfdataset


ds = v5spfdataset(
    # "/Volumes/SPFData/missions/april5/wallarrayv3_2024_04_05_22_13_07_nRX2_rx_circle",
    # "/Volumes/SPFData/missions/april5/wallarrayv3_2024_05_10_05_03_21_nRX2_rx_circle_tag_rand10_90_rand30_100"
    # "/Volumes/SPFData/missions/april5/wallarrayv3_2024_04_10_05_08_55_nRX2_rx_circle",
    # "/Volumes/SPFData/missions/april5/wallarrayv3_2024_05_06_19_04_15_nRX2_bounce",
    # "test_circle2.zarr",
    # f"{repo_root}/test_circle.zarr",
    # "/Volumes/SPFData/missions/april5/wallarrayv3_2024_05_18_16_29_28_nRX2_rx_circle.zarr",
    # "/Volumes/SPFData/missions/april5/wallarrayv3_2024_05_29_20_46_46_nRX2_rx_circle.zarr",
    # "/Volumes/SPFData/missions/april5/wallarrayv3_2024_05_20_11_59_44_nRX2_bounce.zarr",
    # "/Volumes/SPFData/missions/april5/wallarrayv3_2024_06_01_21_14_46_nRX2_rx_circle",
    # "/Users/miskodzamba/Dropbox/research/gits/spf/wallarrayv3_2024_06_02_01_53_23_nRX2_bounce",
    # "/Users/miskodzamba/Dropbox/research/gits/spf/wallarrayv3_2024_06_02_02_24_58_nRX2_bounce",
    # "/Users/miskodzamba/Dropbox/research/gits/spf/wallarrayv3_2024_06_02_02_28_23_nRX2_bounce",
    # "/Users/miskodzamba/Dropbox/research/gits/spf/wallarrayv3_2024_06_02_02_34_03_nRX2_bounce",
    # "/Volumes/SPFData/missions/april5/wallarrayv3_2024_06_02_05_18_27_nRX2_rx_circle",
    # "/Volumes/SPFData/missions/april5/wallarrayv3_2024_06_02_06_47_47_nRX2_rx_circle",
    # "/Volumes/SPFData/missions/april5/allarrayv3_2024_06_02_08_17_31_nRX2_bounce",
    # "/Volumes/SPFData/missions/april5/wallarrayv3_2024_06_04_19_14_43_nRX2_rx_circle",
    # "/Volumes/SPFData/missions/april5/wallarrayv3_2024_06_04_19_44_10_nRX2_bounce",
    # "/Volumes/SPFData/missions/april5/wallarrayv3_2024_06_04_21_10_37_nRX2_bounce",
    # "/Volumes/SPFData/missions/april5/wallarrayv3_2024_06_05_02_41_50_nRX2_rx_circle",
    # "/Volumes/SPFData2/missions/april5/wallarrayv3_2024_06_07_23_43_19_nRX2_bounce",
    # "/mnt/md0/spf/2d_wallarray_v2_data/june/wallarrayv3_2024_06_10_14_00_06_nRX2_rx_circle",
    # "/mnt/md0/spf/2d_wallarray_v2_data/june/wallarrayv3_2024_06_03_01_40_27_nRX2_bounce.zarr",
    # "/mnt/md0/spf/2d_wallarray_v2_data/june/wallarrayv3_2024_06_03_00_30_00_nRX2_rx_circle.zarr",
    "/mnt/md0/spf/2d_wallarray_v2_data/june/wallarrayv3_2024_06_03_01_17_43_nRX2_rx_circle.zarr",
    nthetas=129,
    ignore_qc=True,
)

In [ ]:
ds.get_ground_truth_thetas()

In [ ]:
ds.average_windows_in_segmentation

In [ ]:
np.array(
    [
        [
            len(x["simple_segmentation"]) > 2
            for x in ds.segmentation["segmentation_by_receiver"][f"r{rx_idx}"]
        ]
        for rx_idx in [0, 1]
    ]
).mean()

In [ ]:
session_idx = 150 * 2 + 100
session_idx = 1887
session_idx = len(ds) - 100
session_idx = 5
session_idx = 5
session_idx = 100

In [ ]:
# beam_sds[0].shape, v.shape

# window_size = 2500
# v_windowed = v.reshape(2, -1, window_size).sum(axis=2)
# receiver_idx = 0

# window_sds = beamformer_given_steering_nomean(
#     steering_vectors=ds.steering_vectors[receiver_idx],
#     signal_matrix=v,
# )
# v.shape

In [ ]:
# plt.imshow(window_sds.reshape(65, -1, window_size).mean(axis=2))

In [ ]:
# plt.imshow(windows_sds)

In [ ]:
from spf.rf import beamformer_given_steering_nomean, get_phase_diff, simple_segment
import matplotlib.pyplot as plt

data = ds[session_idx][0]
# signal_matrix = load_zarr_to_numpy(z.receivers["r0"].signal_matrix[session_idx])
n = 2 * 4 * 50000
offset = 0
v = data["signal_matrix"][:, offset : offset + n].numpy()
pd = get_phase_diff(v)

fig, axs = plt.subplots(2, 1, figsize=(12, 6))

axs[0].scatter(np.arange(n), np.abs(v[0]), alpha=0.1, s=1, label="ant0")
axs[0].scatter(np.arange(n), np.abs(v[1]), alpha=0.1, s=1, label="ant1")
axs[0].set_title("Raw signal")
axs[0].legend()
axs[0].set_xlabel("Sample# (time)")
axs[1].set_xlabel("Sample# (time)")
axs[1].set_title("Phase estimates")
axs[1].scatter(np.arange(n), pd, s=1, alpha=0.1)

beam_sds = [
    beamformer_given_steering_nomean(
        steering_vectors=ds.steering_vectors[receiver_idx],
        signal_matrix=v,
    )
    for receiver_idx in range(2)
]

window_sds = []
for window in simple_segment(
    v,
    window_size=2500,
    stride=2500,
    trim=20,
    mean_diff_threshold=0.2,  #
    max_stddev_threshold=0.5,  # just eyeballed this
    drop_less_than_size=3000,
    min_abs_signal=40,
)["simple_segmentation"]:
    if window["type"] == "signal":
        axs[1].plot(
            [window["start_idx"], window["end_idx"]],
            [window["mean"], window["mean"]],
            color="red",
        )
    else:
        axs[1].plot(
            [window["start_idx"], window["end_idx"]],
            [window["mean"], window["mean"]],
            color="orange",
        )
    # print(window["start_idx"], window["end_idx"])
    _beam_sds = beam_sds[0][:, window["start_idx"] : window["end_idx"]].mean(axis=1)
    # _beam_sds = _beam_sds.mean(axis=1)
    # _beam_sds -= _beam_sds.min()
    window_sds.append(_beam_sds)
window_sds = np.array(window_sds)
fig.set_tight_layout(True)

In [ ]:
ds.segmentation["segmentation_by_receiver"]["r0"][0].keys()

In [ ]:
ds[session_idx][0]["windowed_beamformer"][None].shape

In [ ]:
ds[session_idx + 1650][0]["windowed_beamformer"]

In [ ]:
ds[session_idx + 1650][0]["windowed_beamformer"].astype(np.float16)

In [ ]:
plt.imshow(
    ds.segmentation["segmentation_by_receiver"]["r0"][session_idx // 2][
        "windowed_beamformer"
    ]
)

In [ ]:
ds.ground_truth_thetas[0]

In [ ]:
from spf.rf import thetas_from_nthetas


thetas_from_nthetas(65)

In [ ]:
ds.rx_configs[0]

In [ ]:
ds.receiver_data[0]["rx_pos_x_mm"][0], ds.receiver_data[0]["rx_pos_y_mm"][0]

In [ ]:
ds.ground_truth_thetas[1][0] / np.pi

In [ ]:
plt.imshow(window_sds)

In [ ]:
ds[session_idx][0]["simple_segmentation"][0]

In [ ]:
np.array(
    [
        len(x["simple_segmentation"])
        for x in ds.segmentation["segmentation_by_receiver"]["r1"]
    ]
).mean()

In [ ]:
ds.ground_truth_thetas

In [ ]:
from spf.rf import pi_norm


ridx = 0
rx_theta_in_pis = ds.receiver_data[ridx]["rx_theta_in_pis"]
tx_pos = np.array(
    [
        ds.receiver_data[ridx]["tx_pos_x_mm"],
        ds.receiver_data[ridx]["tx_pos_y_mm"],
    ]
)
rx_pos = np.array(
    [
        ds.receiver_data[ridx]["rx_pos_x_mm"],
        ds.receiver_data[ridx]["rx_pos_y_mm"],
    ]
)

# compute the angle of the tx with respect to rx
d = tx_pos - rx_pos

rx_to_tx_theta = np.arctan2(d[0], d[1])
theta = pi_norm(rx_to_tx_theta - rx_theta_in_pis[:] * np.pi)
theta, ds.get_ground_truth_thetas()

In [ ]:
ds.yaml_config

In [ ]:
segmentation = ds.get_segmentation()
mean_phase_results = {}
for receiver, results in segmentation["segmentation_by_receiver"].items():
    mean_phase_results[receiver] = np.array(
        [
            np.array([x["mean"] for x in result["simple_segmentation"]]).mean()
            for result in results
        ]
    )


first_n = 512  # 12 * 8
fig, axs = plt.subplots(1, 1)
axs.scatter(range(first_n), mean_phase_results["r0"][:first_n], s=3, label="Rx0")
axs.scatter(range(first_n), mean_phase_results["r1"][:first_n], s=3, label="Rx1")
axs.legend()
axs.axvline(x=115)
axs.set_title("Mean segmented phase diff")
axs.set_xlabel("Chunk (time)")
axs.set_ylabel("Mean phase diff of seg. chunk")

In [ ]:
mean_phase_results

In [ ]:
# segmentation_by_receiver.keys()

In [ ]:
ds.get_segmentation_mean_phase()
ds.get_estimated_thetas()

In [ ]:
from spf.dataset.spf_dataset import pi_norm
from spf.rf import c as speed_of_light


fig, axs = plt.subplots(1, 2, figsize=(12, 4))

estimated_thetas = ds.get_estimated_thetas()
for rx_idx in [0, 1]:

    axs[rx_idx].scatter(
        range(estimated_thetas[f"r{rx_idx}"][0].shape[0]),
        pi_norm(estimated_thetas[f"r{rx_idx}"][0]),
        s=0.4,
    )
    axs[rx_idx].scatter(
        range(estimated_thetas[f"r{rx_idx}"][1].shape[0]),
        pi_norm(estimated_thetas[f"r{rx_idx}"][1]),
        s=0.4,
    )
    axs[rx_idx].scatter(
        range(estimated_thetas[f"r{rx_idx}"][2].shape[0]),
        pi_norm(estimated_thetas[f"r{rx_idx}"][2]),
        s=0.4,
    )
    axs[rx_idx].set_xlabel("Chunk")
    axs[rx_idx].set_ylabel("estimated theta")

In [ ]:
from spf.dataset.spf_dataset import pi_norm
from spf.rf import reduce_theta_to_positive_y


fig, axs = plt.subplots(1, 2, figsize=(12, 4))

first_n = 3000
estimated_thetas = ds.get_estimated_thetas()
for rx_idx in [0, 1]:
    expected_theta = ds.ground_truth_thetas[rx_idx]
    axs[rx_idx].plot(
        expected_theta[:first_n], alpha=1, color="red", label="ground truth"
    )
    axs[rx_idx].plot(
        reduce_theta_to_positive_y(expected_theta[:first_n]),
        alpha=1,
        color="green",
        label="reduced ground truth",
    )

    n = estimated_thetas[f"r{rx_idx}"][0].shape[0]
    axs[rx_idx].scatter(
        range(first_n),
        pi_norm(estimated_thetas[f"r{rx_idx}"][0])[:first_n],
        s=3,
        label=f"Rx{rx_idx}_peak1",
    )
    axs[rx_idx].scatter(
        range(first_n),
        pi_norm(estimated_thetas[f"r{rx_idx}"][1])[:first_n],
        s=3,
        label=f"Rx{rx_idx}_peak2",
    )
    axs[rx_idx].set_xlabel("Chunk")
    axs[rx_idx].set_ylabel("estimated theta")
    axs[rx_idx].legend()
    axs[rx_idx].set_title(f"Receiver (Rx) {rx_idx}")

In [ ]:
repo_root = "/Users/miskodzamba/Dropbox/research/gits/spf/"
import sys

if repo_root not in sys.path:
    sys.path.append(repo_root)  # go to parent dir

from spf.dataset.spf_dataset import v5spfdataset


ds = v5spfdataset(
    "/Volumes/SPFData/missions/april5/wallarrayv3_2024_05_06_19_04_15_nRX2_bounce",
    nthetas=11,
)

from functools import cache
import gc

from spf.dataset.spf_dataset import v5_collate_beamsegnet, v5_thetas_to_targets
from spf.model_training_and_inference.models.beamsegnet import (
    BeamNSegNetDirect,
    BeamNSegNetDiscrete,
    # BeamNetDirect,
    UNet1D,
    ConvNet,
)

torch_device = torch.device("cpu")
nthetas = 11
lr = 0.001


dataloader_params = {
    "batch_size": 4,
    "shuffle": True,
    "num_workers": 0,
    "collate_fn": v5_collate_beamsegnet,
}
torch.manual_seed(1337)
train_dataloader = torch.utils.data.DataLoader(ds, **dataloader_params)

import random

w = False
if w:

    import wandb

    # start a new wandb run to track this script
    wandb.init(
        # set the wandb project where this run will be logged
        project="projectspf",
        # track hyperparameters and run metadata
        config={
            "learning_rate": lr,
            "architecture": "beamsegnet1",
        },
    )


@cache
def mean_guess(shape):
    return torch.nn.functional.normalize(torch.ones(shape), p=1, dim=1)


X, Y_rad, segmentation = next(iter(train_dataloader))


def batch_to_gt_segmentation(X, Y_rad, segmentation):
    n, _, samples_per_session = X.shape
    window_size = 2048
    stride = 2048
    assert window_size == stride
    assert samples_per_session % window_size == 0
    n_windows = samples_per_session // window_size
    window_status = torch.zeros(n, n_windows)
    for row_idx in range(len(segmentation)):
        for window in segmentation[row_idx]["simple_segmentation"]:
            window_status[
                row_idx,
                window["start_idx"] // window_size : window["end_idx"] // window_size,
            ] = 1
    return window_status[:, None]


def segmentation_mask(X, segmentations):
    seg_mask = torch.zeros(
        X.shape[0], X.shape[2], device=X.device
    )  # X.new(X.shape[0], X.shape[2])
    for row_idx in range(seg_mask.shape[0]):
        for w in segmentations[row_idx]["simple_segmentation"]:
            seg_mask[row_idx, w["start_idx"] : w["end_idx"]] = 1
    return seg_mask[:, None]  # orch.nn.functional.normalize(seg_mask, p=1, dim=1)


# m = BeamNSegNetDiscrete(nthetas=nthetas, symmetry=False).to(torch_device)
# m = BeamNSegNetDirect(nthetas=nthetas, symmetry=False).to(torch_device)
# print("ALL", segmentation[0]["all_windows_stats"].shape)
m = UNet1D().to(torch_device).double()
# m = ConvNet(in_channels=3, out_channels=1, hidden=32)
optimizer = torch.optim.Adam(m.parameters(), lr=0.00001, weight_decay=0)
step = 0
sigmoid = torch.nn.Sigmoid()
X = X.double().to(torch_device)
# X[:, :2] /= 500
for epoch in range(10000):
    # for X, Y_rad, segmentation in train_dataloader:
    if True:
        optimizer.zero_grad()

        # full
        input = X.clone().to(torch_device)
        output = m(input)

        seg_mask = segmentation_mask(X, segmentation)
        print(input.shape, output.shape, seg_mask.shape)

        # downsampled
        # input = torch.Tensor(
        #     np.vstack(
        #         [
        #             segmentation[idx]["all_windows_stats"].transpose()[None]
        #             for idx in range(len(segmentation))
        #         ]
        #     )
        # )
        # input[:, 2] /= 50
        # output = m(input)
        # seg_mask = batch_to_gt_segmentation(X, Y_rad, segmentation)

        loss = ((output - seg_mask) ** 2).mean()
        loss.backward()
        optimizer.step()

        to_log = {"loss": loss.item()}

        _input = input.cpu()
        _output = output.cpu().detach().numpy()
        first_n = 3000

        if step % 1000 == 0:
            print(loss.item())
            fig, axs = plt.subplots(1, 3, figsize=(8, 3))
            s = 0.3
            axs[0].set_title("input (track 0/1)")
            axs[0].scatter(range(first_n), _input[0, 0, :first_n], s=s)
            axs[0].scatter(range(first_n), _input[0, 1, :first_n], s=s)
            axs[1].set_title("input (track 2)")
            axs[1].scatter(range(first_n), _input[0, 2, :first_n], s=s)
            # mw = mask_weights.cpu().detach().numpy()

            axs[2].set_title("output vs gt")
            axs[2].scatter(range(first_n), _output[0, 0, :first_n], s=s)
            axs[2].scatter(
                range(first_n), seg_mask.cpu().detach().numpy()[0, 0, :first_n], s=s
            )
            to_log["fig"] = fig
        if w:
            wandb.log(to_log)
        step += 1


# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

In [ ]:
X.shape

In [ ]:
output.shape, seg_mask.shape

In [ ]:
repo_root = "/Users/miskodzamba/Dropbox/research/gits/spf/"
import sys
import torch

if repo_root not in sys.path:
    sys.path.append(repo_root)  # go to parent dir

from spf.dataset.spf_dataset import v5spfdataset
import matplotlib.pyplot as plt

torch_device = torch.device("cpu")
nthetas = 11
lr = 0.001
batch_size = 8

ds = v5spfdataset(
    "/Volumes/SPFData/missions/april5/wallarrayv3_2024_05_06_19_04_15_nRX2_bounce",
    nthetas=11,
)

from functools import cache
import gc

from spf.dataset.spf_dataset import v5_collate_beamsegnet, v5_thetas_to_targets
from spf.model_training_and_inference.models.beamsegnet import (
    BeamNSegNet,
    BeamNetDirect,
    BeamNetDiscrete,
    ConvNet,
    UNet1D,
)

torch.autograd.set_detect_anomaly(True)


dataloader_params = {
    "batch_size": batch_size,
    "shuffle": True,
    "num_workers": 0,
    "collate_fn": v5_collate_beamsegnet,
}
torch.manual_seed(1337)
train_dataloader = torch.utils.data.DataLoader(ds, **dataloader_params)
w = False
if w:
    import wandb

    # start a new wandb run to track this script
    wandb.init(
        # set the wandb project where this run will be logged
        project="projectspf",
        # track hyperparameters and run metadata
        config={
            "learning_rate": lr,
            "architecture": "beamsegnet1",
        },
    )


def plot_instance(_x, _output_seg, _seg_mask, idx=0):
    fig, axs = plt.subplots(1, 3, figsize=(8, 3))
    s = 0.3
    axs[0].set_title("input (track 0/1)")
    axs[0].scatter(range(first_n), _x[idx, 0, :first_n], s=s)
    axs[0].scatter(range(first_n), _x[idx, 1, :first_n], s=s)
    axs[1].set_title("input (track 2)")
    axs[1].scatter(range(first_n), _x[idx, 2, :first_n], s=s)
    # mw = mask_weights.cpu().detach().numpy()

    axs[2].set_title("output vs gt")
    axs[2].scatter(range(first_n), _output_seg[idx, 0, :first_n], s=s)
    axs[2].scatter(range(first_n), _seg_mask[idx, 0, :first_n], s=s)
    return fig


batch_data = next(iter(train_dataloader))
import pickle

pickle.dump(batch_data, open("test_batch.pkl", "wb"))
skip_segmentation = False
segmentation_level = "downsampled"
if segmentation_level == "full":
    first_n = 10000
    seg_m = UNet1D().to(torch_device)
elif segmentation_level == "downsampled":
    first_n = 256
    seg_m = ConvNet(3, 1, 32, bn=True).to(torch_device)

import torch.nn as nn

beam_m = BeamNetDirect(
    nthetas=nthetas, hidden=16, symmetry=True, other=True, act=nn.SELU, bn=True
).to(torch_device)
# beam_m = BeamNetDiscrete(nthetas=nthetas, hidden=16, symmetry=False).to(torch_device)
m = BeamNSegNet(segnet=seg_m, beamnet=beam_m, circular_mean=True).to(torch_device)

optimizer = torch.optim.AdamW(seg_m.parameters(), lr=0.01, weight_decay=0)

step = 0
head_start = 200
for epoch in range(10000):
    if step == head_start:
        optimizer = torch.optim.AdamW(beam_m.parameters(), lr=0.001, weight_decay=0)
        optimizer.zero_grad()
    # for X, Y_rad in train_dataloader:
    optimizer.zero_grad()

    # copy to torch device
    if segmentation_level == "full":
        x = batch_data["x"].to(torch_device)
        y_rad = batch_data["y_rad"].to(torch_device)
        seg_mask = batch_data["segmentation_mask"].to(torch_device)
    elif segmentation_level == "downsampled":
        x = batch_data["all_windows_stats"].to(torch_device)
        y_rad = batch_data["y_rad"].to(torch_device)
        seg_mask = batch_data["downsampled_segmentation_mask"].to(torch_device)
    else:
        raise NotImplementedError

    assert seg_mask.ndim == 3 and seg_mask.shape[1] == 1

    # run beamformer and segmentation
    if not skip_segmentation:
        output = m(x)
    else:
        output = m(x, seg_mask)

    # x to beamformer loss (indirectly including segmentation)
    x_to_beamformer_loss = -beam_m.loglikelihood(output["pred_theta"], y_rad)
    assert x_to_beamformer_loss.shape == (batch_size, 1)
    x_to_beamformer_loss = x_to_beamformer_loss.mean()

    # segmentation loss
    x_to_segmentation_loss = (output["segmentation"] - seg_mask) ** 2
    assert x_to_segmentation_loss.ndim == 3 and x_to_segmentation_loss.shape[1] == 1
    x_to_segmentation_loss = x_to_segmentation_loss.mean()

    if skip_segmentation:
        loss = x_to_beamformer_loss
    else:
        if step >= head_start:
            loss = x_to_beamformer_loss
        else:
            loss = x_to_segmentation_loss
    # if step in [799, 780]:
    #     print(step, output)
    loss.backward()
    optimizer.step()

    to_log = {
        "loss": loss.item(),
        "segmentation_loss": x_to_segmentation_loss.item(),
        "beam_former_loss": x_to_beamformer_loss.item(),
    }
    if step % 500 == 0:
        # beam outputs
        img_beam_output = (
            (beam_m.render_discrete_x(output["pred_theta"]) * 255).cpu().byte()
        )
        img_beam_gt = (beam_m.render_discrete_y(y_rad) * 255).cpu().byte()
        train_target_image = torch.zeros(
            (img_beam_output.shape[0] * 2, img_beam_output.shape[1]),
        ).byte()
        for row_idx in range(img_beam_output.shape[0]):
            train_target_image[row_idx * 2] = img_beam_output[row_idx]
            train_target_image[row_idx * 2 + 1] = img_beam_gt[row_idx]
        if w:
            output_image = wandb.Image(
                train_target_image, caption="train vs target (interleaved)"
            )
            to_log["output"] = output_image

        # segmentation output
        _x = x.detach().cpu().numpy()
        _seg_mask = seg_mask.detach().cpu().numpy()
        # _output_seg = output_segmentation_upscaled.detach().cpu().numpy()
        _output_seg = output["segmentation"].detach().cpu().numpy()

        fig = plot_instance(_x, _output_seg, _seg_mask, idx=0)
        if w:
            to_log["fig"] = fig
    if w:
        wandb.log(to_log)
    else:
        # if step > 760 and step < 800:
        if step % 20 == 0:
            print(
                step,
                loss.item(),
                x_to_beamformer_loss.item(),
                x_to_segmentation_loss.item(),
            )
    step += 1

# [optional] finish the wandb run, necessary in notebooks
if w:
    wandb.finish()

In [ ]:
import torch.nn as nn

z = nn.Sigmoid()

In [ ]:
z(torch.Tensor([-10, 0, 10]))

In [ ]:
weighted_input = torch.mul(x, output["segmentation"]).sum(axis=2) / output[
    "segmentation"
].sum(axis=2)

In [ ]:
param = m.beamnet.fixify(m.beamnet.beam_net(weighted_input), sign=1)

m.beamnet.likelihood(param, y_rad)

In [ ]:
param, y_rad

In [ ]:
_param = param.clone()
_param[:, 1] = 100
m.beamnet.likelihood(_param, y_rad)

In [ ]:
y_rad.clamp(min=0, max=0.1)
y_rad

In [ ]:
m.beamnet.beam_net(weighted_input)

In [ ]:
m.beamnet.beam_net(weighted_input)

In [ ]:
output["pred_theta"]

In [ ]:
weighted_input = torch.mul(x, output["segmentation"]).sum(axis=2) / output[
    "segmentation"
].sum(axis=2)
weighted_input.shape

In [ ]:
(output["pred_theta"][:, 0] - y_rad).shape, x_to_beamformer_loss.shape

In [ ]:
beam_m.loglikelihood(output["pred_theta"], y_rad).shape

In [ ]:
torch.mul(output["beam_former"], output["segmentation"]).sum(axis=2) / output[
    "segmentation"
].sum(axis=2)

In [ ]:
x = output["pred_theta"]
y = y_rad
(x[:, 3] * torch.exp(-((x[:, 0] - y) ** 2) / x[:, 1])).shape

In [ ]:
output["pred_theta"].shape

In [ ]:
output["beam_former"].shape

In [ ]:

    output_segmentation_upscaled = output["segmentation"] * seg_mask.sum(
        axis=2, keepdim=True
    )
    x_to_segmentation_loss = (output_segmentation_upscaled - seg_mask) ** 2

In [ ]:
output["mask_weights"].shape, output["segmentation"].shape, output["beam_former"].shape
k = torch.mul(output["beam_former"], output["segmentation"]) / output[
    "segmentation"
].sum(axis=2, keepdim=True)
k.shape

In [ ]:
# output_segmentation_upscaled = output["segmentation"] * seg_mask.sum()
# x_to_segmentation_loss = (output_segmentation_upscaled - seg_mask) ** 2
(output["segmentation"] * seg_mask.sum(axis=2, keepdim=True)).sum(axis=2)

In [ ]:
seg_mask.sum(axis=2, keepdim=True)

In [ ]:
z = output["segmentation"].detach().cpu().numpy()[0, 0]
# =_p_seg_mask[0,0]
# z=_output_seg[0,0]
plt.scatter(range(len(z)), z)

In [ ]:
output["segmentation"].shape

In [ ]:
X[:, 1, :].mean(), X[:, 1, :].std()

In [ ]:
Y_rad

In [ ]:
output.shape, Y_rad.shape

In [ ]:
segmentation[0]["all_windows_stats"].shape

In [ ]:
segmentation_mask(X, segmentation)

In [ ]:
m(X)

In [ ]:
_X = X.clone().to(torch_device)
_X[:, :2] /= 500
batch_size, input_channels, session_size = _X.shape
beam_former_input = _X.transpose(1, 2).reshape(
    batch_size * session_size, input_channels
)
print(_X.device, beam_former_input)
beam_former = m.beam_net(beam_former_input).reshape(
    batch_size, session_size, 5  # mu, o1, o2, k1, k2
)
mask_weights = m.softmax(m.unet1d(_X)[:, 0])

In [ ]:
beam_former_input

In [ ]:
ds[0]

In [ ]:
seg_mask.sum(axis=1)

In [ ]:
seg_mask.cpu().detach().numpy()[0].sum()

In [ ]:
first_n = 40000
x = X[0].cpu()

fig, axs = plt.subplots(1, 3, figsize=(12, 4))
axs[0].scatter(range(first_n), x[0, :first_n], s=0.3)
axs[0].scatter(range(first_n), x[1, :first_n], s=0.3)
axs[1].scatter(range(first_n), x[2, :first_n], s=0.3)
# mw = mask_weights.cpu().detach().numpy()
mw = m(X).cpu().detach().numpy()[0]
axs[2].scatter(range(first_n), mw[0, :first_n], s=0.3)
axs[2].scatter(range(first_n), seg_mask.cpu().detach().numpy()[0, :first_n], s=0.3)

In [ ]:
mask_weights[0]

In [ ]:
from spf.model_training_and_inference.models.beamsegnet import BeamNSegNetDirect


m = BeamNSegNetDirect(nthetas=nthetas)

optimizer = torch.optim.AdamW(m.parameters(), lr=0.01)

m.beam_net.beam_net[0].weight.grad

In [ ]:
x, y = next(iter(train_dataloader))

In [ ]:
k = x[[0]]
k_y = y[[0]]
k[:, 2] = -k[:, 2].sign() * k[:, 2]
# k[:, 2] = k[:, 2].sign() * k[:, 2]

In [ ]:
k[:, 2]

In [ ]:
optimizer.zero_grad()
m.train()
m.beam_net.beam_net[0].weight.grad

In [ ]:
X.max()

In [ ]:
output = m(k)

loss_fn = torch.nn.MSELoss()
l = loss_fn(output, k_y)
l.backward()
# mean_loss = output
# optimizer.step()

In [ ]:
output

In [ ]:
m.beam_net.beam_net[0].weight.grad

In [ ]:
plt.imshow(Y.to("cpu"))